In [2]:
!pip install shamirs


'pip' is not recognized as an internal or external command,
operable program or batch file.


supaya bisa diakses, perlu persetujuan dari beberapa orang untuk combine kuncinya.

In [8]:
import shamirs

# Rahasia yang ingin dibagi (misalnya, dalam bentuk integer)
# secret =  b"myPassword123"
secret =  b"myPassword123"
# Konversi bytes ke integer
secret_int = int.from_bytes(secret, byteorder="big")

# Jumlah bagian (shares) yang ingin dibuat
num_shares = 5

# Jumlah minimal shares yang diperlukan untuk merekonstruksi rahasia
threshold = 3

# Membagi rahasia menjadi shares
shares = shamirs.shares(secret_int, num_shares, threshold=threshold)

In [4]:
# Menampilkan shares yang dihasilkan
print("Shares:")
for share in shares:
    print(f"Index: {share.index}, Value: {share.value}")


Shares:
Index: 1, Value: 140975672365782519455444503600935192703
Index: 2, Value: 91016494379843454918959295566514786419
Index: 3, Value: 20263658176066612630241049164460821006
Index: 4, Value: 98858347214921224320977068110657402191
Index: 5, Value: 156659378035938058259480048689220424247


In [5]:
# Misalkan kita memiliki sejumlah shares yang memenuhi threshold
selected_shares = shares[0:3]

# Merekonstruksi rahasia dari shares yang dipilih
recovered_secret_int = shamirs.interpolate(selected_shares)

print(f"Recovered Secret: {recovered_secret_int}")

# Konversi integer kembali ke bytes
recovered_secret = recovered_secret_int.to_bytes((recovered_secret_int.bit_length() + 7) // 8, byteorder="big")

print(f"Recovered Secret: {recovered_secret}")

Recovered Secret: 8673414574508009369551837934131
Recovered Secret: b'myPassword123'


#### Konversi Password menggunakan hash function, lalu hasilnya dibagi menjadi 7 bagian dan hanya jika 5 bagian terkumpul baru bisa membaca isi pesan rahasia

In [17]:
import shamirs
import os, base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

password = "Halo aku adalah manusia".encode()
# password = "myPassword123".encode()
salt = os.urandom(16) # Buat salt acak (16 bytes)

# Gunakan PBKDF2HMAC untuk membuat hash
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
)

hashed_password = kdf.derive(password)

print(hashed_password)
print("Salt (Base64):", base64.b64encode(salt).decode())
print("Derived Hash (Base64):", base64.b64encode(hashed_password).decode())
print()

# Konversi bytes ke integer
PRIME = 2**127 - 1
secret_int = int.from_bytes(hashed_password, byteorder="big") % PRIME

# Jumlah bagian (shares) yang ingin dibuat
num_shares = 7

# Jumlah minimal shares yang diperlukan untuk merekonstruksi rahasia
threshold = 5

# Membagi rahasia menjadi shares
shares = shamirs.shares(secret_int, num_shares, threshold=threshold)
print()

# Menampilkan shares yang dihasilkan
print("Shares:")
for share in shares:
    print(f"Index: {share.index}, Value: {share.value}")
print()

# Misalkan kita memiliki sejumlah shares yang memenuhi threshold
selected_shares = shares[0:6]

# Merekonstruksi rahasia dari shares yang dipilih
recovered_secret_int = shamirs.interpolate(selected_shares)

print(f"Recovered Secret: {recovered_secret_int}")

# Konversi integer kembali ke bytes
recovered_secret = recovered_secret_int.to_bytes((recovered_secret_int.bit_length() + 7) // 8, byteorder="big")

print(f"Recovered Secret: {recovered_secret}")

b'\xf2\x8d\xea\xb1\xa3\xf6\xe8\xc8B/\xd0\xab\xbd\xb4\x9e%\xf1\x14\xfb0\x9c\x82\xfbU\xb1\x89\xdd\xde\xb4\xb2\x82\x0c'
Salt (Base64): WxAQigNLeTIElR3lnIG06w==
Derived Hash (Base64): 8o3qsaP26MhCL9CrvbSeJfEU+zCcgvtVsYnd3rSyggw=


Shares:
Index: 1, Value: 81168104556367084681750953292916799292
Index: 2, Value: 59958049946932384456255491176437771081
Index: 3, Value: 79956715247279005957501062143329080461
Index: 4, Value: 68720281740735085059309482418731585243
Index: 5, Value: 22482484723991854087688850485118818253
Index: 6, Value: 6154613507103641820833547082296987332
Index: 7, Value: 153325511413487873489124235207404975336

Recovered Secret: 114567068345148835480664610809073876571
Recovered Secret: b'V0\xd0\x93\xe4p\xcc\xe65\xe9\x7f60\x1b\xbe['
